In [1]:
# import os
# from pydub import AudioSegment

# input_root = "audio"
# output_root = "audio_wav"

# for subdir in os.listdir(input_root):
#     input_subdir = os.path.join(input_root, subdir)
#     output_subdir = os.path.join(output_root, subdir)

#     if not os.path.isdir(input_subdir):
#         continue

#     os.makedirs(output_subdir, exist_ok=True)

#     for filename in os.listdir(input_subdir):
#         if filename.endswith(".mp3"):
#             input_path = os.path.join(input_subdir, filename)
#             output_path = os.path.join(output_subdir, filename.replace(".mp3", ".wav"))

#             try:
#                 audio = AudioSegment.from_mp3(input_path)
#                 audio = audio.set_channels(1).set_frame_rate(32000)
#                 audio.export(output_path, format="wav")
#             except Exception as e:
#                 print(f"⚠️ Failed to convert: {input_path}")
#                 print(f"   Reason: {e}")


In [2]:
import pandas as pd

# Load the uploaded CSV file
csv_path = "./annotations_final.csv"
df = pd.read_csv(csv_path, sep="\t")

# Display the first few rows to understand its structure
df.head(10)

,clip_id,no voice,singer,duet,plucking,hard rock,world,bongos,harpsichord,female singing,...,rap,metal,hip hop,quick,water,baroque,women,fiddle,english,mp3_path
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
1,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
2,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
3,11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
4,12,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
5,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,c/lvx_nova-lvx_nova-01-contimune-30-59.mp3
6,19,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,c/lvx_nova-lvx_nova-01-contimune-175-204.mp3
7,21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,c/lvx_nova-lvx_nova-01-contimune-233-262.mp3
8,23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,c/lvx_nova-lvx_nova-01-contimune-291-320.mp3
9,25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0/american_bach_soloists-j_s__bach__cantatas_v...


In [3]:
import json

# Get the tag columns (all except clip_id and mp3_path)
tag_columns = df.columns.difference(['clip_id', 'mp3_path'])

# Output list
samples = []

for _, row in df.iterrows():
    tags = [tag.replace("_", " ") for tag in tag_columns if row[tag] == 1]
    if not tags:
        continue  # skip samples with no tags
    
    # Generate simple natural language prompt
    prompt = f"A music clip with " + ", ".join(tags[:-1]) + (" and " + tags[-1] if len(tags) > 1 else tags[0]) + "."

    samples.append({
        "audio_filepath": f"audio_wav/{row['mp3_path'].replace('.mp3', '.wav')}",
        "text": prompt
    })

# Save to JSONL
jsonl_path = "./audio_wav/magnatagatune_text_audio_pairs.jsonl"
with open(jsonl_path, "w") as f:
    for sample in samples:
        f.write(json.dumps(sample) + "\n")

jsonl_path

'./audio_wav/magnatagatune_text_audio_pairs.jsonl'

In [ ]:
import torch
import torchaudio
from encodec import EncodecModel
from encodec.utils import convert_audio
import json

# 加载模型
model = EncodecModel.encodec_model_32khz()
model.eval()

# 输入 jsonl 文件
input_jsonl = "magnatagatune_text_audio_pairs.jsonl"
output_jsonl = "magnatagatune_tokenized.jsonl"

with open(input_jsonl, "r") as fin, open(output_jsonl, "w") as fout:
    for line in fin:
        item = json.loads(line)
        audio_path = item["audio_filepath"]

        # 加载并预处理音频
        wav, sr = torchaudio.load(audio_path)
        wav = convert_audio(wav, sr, model.sample_rate, model.channels)
        
        # 提取离散 token
        with torch.no_grad():
            encoded_frames = model.encode(wav)
            codes = [code.cpu().numpy().tolist() for code in encoded_frames[0].codes]  # shape: (K, T)

        item["audio_tokens"] = codes
        fout.write(json.dumps(item) + "\n")
